In [1]:
import requests
import lxml.html as lh
import pandas as pd
import re

In [12]:
# downloading and saving canada tours and tickets page
def scrape(url):
    page = requests.get(url)
    html = page.content
    open('canada_tours.txt','wb').write(html)
    categories()

In [13]:
def get_tours(tour_category):
    html = open("htmlpages/"+tour_category+".txt",'r',encoding='utf-8').read()
    tree = lh.fromstring(html)
    
    attractions = tree.xpath('//div[@class="listing attraction_element"]//div[@class="listing_title"]/a/@onclick')    
    attractions = [base_page+attraction.split(';')[-2].split(',')[2][2:-1] for attraction in attractions]
   
    next_attractions = list()
    if tree.xpath("//div[@class='unified pagination ']"):
        next_page_container = tree.xpath("//div[@class='unified pagination ']")[0]
        if next_page_container.xpath("./span/@class") != ["nav next disabled"]:
            next_page = base_page+next_page_container.xpath("./a[@class='nav next rndBtn ui_button primary taLnk']/@href")[0]
            page_no = next_page_container.xpath("./a[@class='nav next rndBtn ui_button primary taLnk']/@data-page-number")[0]
            page = requests.get(next_page)
            if bool(re.search(r'\d',tour_category)):
                file_name = re.sub(r'\d+','',tour_category)+str(page_no)
            else:
                file_name = tour_category+str(page_no)
            open("htmlpages/"+file_name+".txt",'wb').write(page.content)
            next_attractions = get_tours(file_name)
    
    if len(next_attractions) != 0:
        attractions += next_attractions
    
    return attractions

In [14]:
base_page = "https://tripadvisor.ca"
scrape('https://www.tripadvisor.ca/Attraction_Products-g153339-Canada.html')

ParserError: Document is empty